In [4]:
\cd ../qenv

In [5]:
\l test.q

oId price offset lqty dqty state
--------------------------------
1         0                1    
2         208              0    
oId price offset lqty dqty state
--------------------------------
1         0                1    
2         208              0    
"###################################################################################################"
"                                             TEST"
"###################################################################################################"
"==================================================================================================="
"31) .engine.logic.order.New (UNIT) :READY"
`Global function for processing new orders, amending orders and cancelling orders (amending to 0)
"---------------------------------------------------------------------------------------------------"
caseId state dscr                                                                                                    
----------------

In [6]:
setup:((!) . flip(
                (`instrument;(`iId`cntTyp`faceValue`mkprice`smul;enlist(0;0;1;1000;1))); 
                (`account;(`aId`avail`bal`lng`srt`ft`rt`wit`time`froz;enlist(0;0;0;(0 1);(0 -1);0;0;0;z;0))); 
                (`inventory;(`aId`side`mm`upnl`ordQty`ordLoss`amt;flip(0 0;-1 1;0 0;0 0;0 0;0 0;10 10))); 
                (`feetier;(`ftId`vol`bal`ref;flip(0 1;0 0;0 0;0 0))); // Update Account
                (`orderbook;(
                        `iId`price`side`qty`hqty`iqty`vqty`time;
                        flip(0 0;1000 1001;2#1;2#1000;2#0;2#0;2#200;2#z)));
                (`order;(
                        `oId`aId`iId`ivId`side`lqty`oqty`price`reduce`dqty`okind`state`offset;
                        flip(1 2;0 0;0 0;((0 1);(0 1));1 1;200 200;200 200;1000 1000;11b;1 1;1 1;0 0;0 210))
                );
                (`risktier;(`rtId`amt`lev;flip(0 1;50000 250000;125 100))) // Update Account
            ));

In [7]:
.engine.testutils.SwitchSetupModels[setup];

In [8]:
x:.model.Order . (`oId`aId`iId`ivId`side`oqty`lqty`price`reduce`dqty;flip(4 5;0 0;0 0;((0 1);(0 1));1 1;300 100;300 100;1000 1000;11b;300 100));

In [9]:
l:0!?[`.engine.model.orderbook.Orderbook;(
        (in;`side;x`side);
        (>;(+;`qty;(+;`hqty;(+;`iqty;`vqty)));0);
        (|;(<;(+\;`qty);sum[x`oqty]);
        (=;`i;(*:;`i))));0b;()];

In [10]:
aqty:sum[l[`qty`iqty`hqty`vqty]];
thresh:sums[aqty];
rp:min[(sum x`oqty;first[aqty])]^((thresh-prev[thresh])-(thresh-sum x`oqty));

In [11]:
o:?[`.engine.model.order.Order;(
            (=;`okind;1);
            (in;`price;l[`price] where (rp>0));
            (in;`state;(0 1));(>;`oqty;0));0b;()];

oId| aId iId ivId price slprice side sprice trig tif okind oskind reduce state oqty dqty lqty slqty offset einst time
---| ----------------------------------------------------------------------------------------------------------------
1  | 0   0   1    1000          1                    1            1      0     200  1    200        0                
2  | 0   0   1    1000          1                    1            1      0     200  1    200        210              


In [13]:
s:0!((`price`side`iId xkey l) lj (`price`side`iId xgroup o));
msk:raze[.util.PadM[{x#1}'[count'[s`oId]]]];

s[`aqty]:aqty;
s[`thresh]:thresh;
s[`rp]:rp;
// Pad s into a matrix
// for faster operations
//pdcl:`oId`side`price`offset`oqty`lqty`reduce`aId`state;
//(s pdcl):.util.PadM'[s pdcl];

// Useful counts 
s[`maxN]:max count'[s`offset];
s[`tmaxN]:til'[s`maxN];
s[`numLvls]:count[s`offset];

In [14]:
flip s

price  | 1000   
side   | 1      
iId    | 0      
qty    | 1000   
hqty   | 0      
iqty   | 0      
vqty   | 200    
time   |        
oId    | 1 2    
aId    | 0 0    
ivId   | 1 1    
slprice|        
sprice |        
trig   |        
tif    |        
okind  | 1 1    
oskind |        
reduce | 11b    
state  | 0 0    
oqty   | 200 200
dqty   | 1 1    
lqty   | 200 200
slqty  |        
offset | 0 210  
einst  |        
aqty   | 1200   
thresh | 1200   
rp     | 400    
maxN   | 2      
tmaxN  | 0 1    
numLvls| 1      


In [51]:
/ hqty:.util.Clip[(-/)(hqty,rp)]
/ nlqty:.util.Clip[{?[x>z;(y+z)-x;y]}'[rp;lqty;offset]]

In [15]:
so:ungroup s

In [ ]:
//agent fills is set to orders?
.engine.model.order.Order:([oId:`long$()] 
        aId:`.engine.model.account.Account$();iId:`.engine.model.instrument.Instrument$();
        ivId:`.engine.model.inventory.Inventory$();price:`long$();slprice:`long$();side:`long$();
        sprice:`long$();trig:`long$();tif:`long$();okind:`long$();oskind:`long$();reduce:`boolean$();
        state:`long$();oqty:`long$();dqty:`long$();lqty:`long$();slqty:`long$();offset:`long$();
        einst:`long$();time:`datetime$());

In [16]:
.event.Order:{x:.event.prep[x];([]kind:`order;time:x`time;datum:flip x`oId`time`aId`side`okind`price`slqty`lprice`slprice`state`reduce`trig`einst;aId:7h$x`aId)}

In [2]:
\c 600 600

In [3]:
parse"update 
    iId:iId,
    oId:oId,
    time:time,
    aId:aId,
    side:side,
    okind:okind,
    price:price,
    lqty:max[(?[rp>lqty;(lqty+offset)-rp;lqty];0)],
    dqty:max[(?[((dqty<lqty) and (lqty>0));dqty;lqty])], // todo dependent
    offset:max[(offset-rp;0)],  // todo dependent
    shft:lqty+offset,  // todo dependent
    nh:((offset<=rp)and((lqty+offset)<=rp))
    state:?[((offset<=rp)and((lqty+offset)<=rp));1;?[(sums[offset]<=rp);2;state]],  // todo dependent
    reduce:reduce,
    slprice:slprice,
    slqty:slqty,
    state:state,
    reduce:reduce,
    trig:trig,
    einst:einst
    from so"

!
`so
()
0b
`iId`oId`time`aId`side`okind`price`lqty`dqty`offset`shft`nh`reduce`slprice`slqty`state`reduce1`trig`einst!(`iId;`oId;`time;`aId;`side;`okind;`price;(max;(enlist;(?;(>;`rp;`lqty);(-;(+;`lqty;`offset);`rp);`lqty);0));(max;(?;(&;(<;`dqty;`lqty);(>;`lqty;0));`dqty;`lqty));(max;(enlist;(-;`offset;`rp);0));(+;`lqty;`offset);((&;((';~:;>);`offset;`rp);((';~:;>);(+;`lqty;`offset);`rp));(:;`state;(?;(&;((';~:;>);`offset;`rp);((';~:;>);(+;`lqty;`offset);`rp));1;(?;((';~:;>);(+\;`offset);`rp);2;`state))));`reduce;`slprice;`slqty;`state;`reduce;`trig;`einst)


In [17]:
no:flip update 
    iId:iId,
    oId:oId,
    time:time,
    aId:aId,
    side:side,
    okind:okind,
    price:price,
    lqty:max[(?[rp>lqty;(lqty+offset)-rp;lqty];0)],
    dqty:max[(?[((dqty<lqty) and (lqty>0));dqty;lqty])], // todo dependent
    offset:max[(offset-rp;0)],  // todo dependent
    shft:lqty+offset,  // todo dependent
    nh:((offset<=rp)and((lqty+offset)<=rp))
    state:?[((offset<=rp)and((lqty+offset)<=rp));1;?[(sums[offset]<=rp);2;state]],  // todo dependent
    reduce:reduce,
    slprice:slprice,
    slqty:slqty,
    state:state,
    reduce:reduce,
    trig:trig,
    einst:einst
    from so

In [18]:
no

price  | 1000 1000
side   | 1    1   
iId    | 0    0   
qty    | 1000 1000
hqty   | 0    0   
iqty   | 0    0   
vqty   | 200  200 
time   |          
oId    | 1    2   
aId    | 0    0   
ivId   | 1    1   
slprice|          
sprice |          
trig   |          
tif    |          
okind  | 1    1   
oskind |          
reduce | 1    1   
state  | 0    0   
oqty   | 200  200 
dqty   | 1    1   
lqty   | 0    10  
slqty  |          
offset | 0    0   
einst  |          
aqty   | 1200 1200
thresh | 1200 1200
rp     | 400  400 
maxN   | 2    2   
tmaxN  | 0    1   
numLvls| 1    1   
shft   | 200  410 
nh     | 0    0   
reduce1| 1    1   


In [20]:
nl:flip select
   hqty:max[(hqty-rp;0)],
   iqty:lqty-dqty,
   vqty:0,
   qty:0,
   iId:iId,
   time:time,
   price:price
   from s

In [21]:
nl

hqty | 0      
iqty | 199 199
vqty | 0      
qty  | 0      
iId  | 0      
time |        
price| 1000   


In [ ]:
f:?[`]

In [22]:
f:select //book fills
    time:time,
    price:price,
    qty:lqty - lqty, // todo dependency
    reduce:reduce,
    ismaker:okind=1,
    side:side,
    iId:iId,
    oId:oId,
    ivId:ivId,
    aId:aId
    from so

In [23]:
f

time price qty reduce ismaker side iId oId ivId aId
---------------------------------------------------
     1000  0   1      1       1    0   1   1    0  
     1000  0   1      1       1    0   2   1    0  


In [31]:
t:select from (ungroup select 
    price:price,
    side:side,
    bqty:qty,
    qty:flip(offset[;0]-hqty;oqty[;0];(0^(offset[;1]));oqty[;1];qty) // todo fix
    from s) where qty>0

In [34]:
t

price side bqty qty 
--------------------
1000  1    1000 200 
1000  1    1000 210 
1000  1    1000 200 
1000  1    1000 1000


In [ ]:
nhqty:          .util.Clip[(-/)s`hqty`rp]; // Derive the new hidden qty
nlqty:          .util.Clip[{?[x>z;(y+z)-x;y]}'[s`rp;s`lqty;s`offset]]; // TODO faster/fix
ndqty:          .util.Clip[{?[((x<y) and (y>0));x;y]}'[s[`dqty];nlqty]]; // TODO faster/fix
niqty:          sum'[nlqty-ndqty]; // The new order invisible qty = lqty - display qty
displaydlt:     (ndqty-s[`dqty]); // Derive the change in the order display qty
lqtydlt:        (nlqty-s[`lqty]); // Derive the change in the order lqty
hqtydlt:        (nhqty-s[`hqty]); // Derive the change in hidden order qty
noffset:        .util.Clip[((-/)s`offset`rp)]; // Subtract the replaced amount and clip<0 (offset includes hqty)
nqty:           .util.Clip[((-/)s`qty`rp)-(sum'[lqtydlt]+hqtydlt)]; // qty -rp - qty attributed to other qtys
nvqty:          nqty+sum'[ndqty]; // Derive the new visible qty as order display qty + the new qty from above
nshft:          nlqty+noffset; // 
nmxshft:        {$[x>1;max[y];x=1;y;0]}'[maxN;nshft]; // the max shft for each price TODO make faster
nfilled:        s[`oqty] - nlqty; // New amount that is filled (in total)
accdlts:        s[`lqty] - nlqty; // The new amounts that will attribute to fills.
nobupd:         count[nhqty];